In [50]:
import os
import yaml
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from typing import List, TypedDict, Annotated
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import tool
"""
from langchain.vectorstores import FAISS
from langchain_core.graph import Graph, Node, StartNode, StopNode
from langchain.docstore.document import Document
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
"""

# Load environment variables
load_dotenv()
api_key = os.environ["groq"]

# Initialize LLM
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=api_key
)
llm_ds= ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
    api_key=api_key
    # other params...
)

In [2]:
llm.invoke("hi").content

"It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [3]:
llm.invoke("where is taj mahal?").content

"The Taj Mahal is located in Agra, a city in the Indian state of Uttar Pradesh. It is situated on the southern bank of the Yamuna River, approximately 230 kilometers (143 miles) south of New Delhi, the capital city of India.\n\nThe exact address of the Taj Mahal is:\n\nTaj Mahal, Dharmapuri, Forest Colony, Tajganj, Agra, Uttar Pradesh 282001, India\n\nIt's one of the most famous and iconic landmarks in India, and a popular tourist destination, attracting millions of visitors each year."

In [24]:
resume,personal_info="","""Strategic and results-driven talent acquisition leader with a proven track record in recruiting and talent management across diverse industries. I specialize in designing and executing scalable recruitment strategies that align with organizational goals and elevate the candidate experience.

With a deep understanding of data-driven hiring, I leverage analytics to streamline processes, enhance team performance, and drive informed decision-making. I’m not just focused on filling roles—I build and lead high-performing recruitment teams capable of managing high-volume, cross-functional hiring with precision and efficiency.

My approach has consistently improved hiring outcomes, aligning recruitment operations with business objectives while ensuring an exceptional stakeholder experience. Backed by a strong foundation in Customer Relationship Management and Human Resources, I am passionate about delivering excellence in every aspect of talent acquisition."""
with open('/workspaces/github_analysis/data/job_description.txt', 'r') as file:
    job_description=file.read()
loader = PyPDFLoader("/workspaces/github_analysis/data/resume_with_capg.pdf")
documents = loader.load()
for i in documents:
    resume+=(i.page_content.strip())
    
with open('/workspaces/github_analysis/src/prompts.yaml', 'r') as file:
    data = yaml.safe_load(file)
    
def intro():

    print("*"*100)
    for i,j in enumerate(data.keys()):
        print(f"{i+1}. {j}")
def inputs():
    i=input("Enter the number corresponding to the key you want to use: ")
    print("selected number is ",int(i))
    key=list(data.keys())[int(i)-1]
    return key

def func(key):
    input_variables=list(data[key]["inputs"].keys())
    template=data[key]["prompt"].format(resume=resume, job_description=job_description,personal_info=personal_info)
    prompt_template = PromptTemplate(
        input_variables=input_variables,
        template=template
    )
    print(prompt_template,input_variables)
    chain=prompt_template | llm
    response=chain.invoke(data[key]["inputs"]).content
    return response

In [66]:
intro()


****************************************************************************************************
1. resume_and_job_decription_rewrite
2. cold_mail_and_person_info
3. cold_mail_no_personal_info


In [67]:
x=inputs()

selected number is  2


In [26]:
print(func(x))

input_variables=[] input_types={} partial_variables={} template='"""\nUsing the following resume, write a professional and inquisitive cold mail tailored to the provided person\'s information, seeking job opportunities:\n\nResume: Dinesh\tSagar\n+91-8121909150\n\t\nMail\n\t\nGitHub\n\t\nLinkedIn\tProfile\nPassionate\tData\tScientist\twith\ta\tdrive\tto\ttackle\treal-world\tchallenges\tthrough\tdata\tanalysis\tand\tmachine\tlearning.\nCommitted\tto\textracting\tvalue\tfrom\tcomplex\tdata\tand\tdelivering\tactionable\tinsights\tto\tinform\tbusiness\tdecisions\t\nand\ndrive\tgrowth.\tEager\tto\tcontinuously\tlearn\tand\timprove\tto\tdeliver\timpactful\tsolutions.\nExperience\nCapgemini\nConsultant\nFEB\t2025-\tPRESENT\nAmazon\tDevelopment\tCentre\tIndia\nQuality\tSpecialist\nJAN\t2020\t-\tDEC\t2024\nImplemented\tan\toutlier\tdetection\tprogram\tto\nenhance\taddress\tdata\tquality,\timproving\tlogistics\nperformance\tfor\tAmazon\tLogistics.\nAnalyzed\tlarge-scale\tdatasets\tusing\tstatisti

In [7]:
import os

In [8]:
os.getcwd()

'/workspaces/github_analysis/research'

In [19]:
os.chdir("/workspaces/github_analysis/")
from src.helpers import *

In [23]:
data

{'resume_and_job_decription_rewrite': {'tool_use_case': 'use this tool to rewrite a resume based on a job description provided.',
  'prompt': '"""\nRewrite the following resume to match the job description provided:\n\nResume: {resume}\n\nJob Description: {job_description}\n\nInstructions:\n1) Use the available experience, projects, and skills to rewrite the resume.\n2) Don\'t Halucinate.\n3) Don\'t use any external data but only the resume and job description.\n4) Don\'t mention any comments and preamble in the response.\n"""\n',
  'inputs': {'resume': 'resume', 'job_description': 'job_description'}},
 'cold_mail_and_person_info': {'tool_use_case': "use this tool to write a professional and inquisitive cold mail tailored to a person's information, seeking job opportunities.",
  'prompt': '"""\nUsing the following resume, write a professional and inquisitive cold mail tailored to the provided person\'s information, seeking job opportunities:\n\nResume: {resume}\n\nPerson Information: {

In [ ]:
"""
creating tools and binding them to the LLM

"""

da


In [26]:
data.keys()

dict_keys(['resume_and_job_decription_rewrite', 'cold_mail_and_person_info', 'cold_mail_no_personal_info'])

In [59]:
from langchain_core.tools import tool

def make_tool_func(key):
    code = f'''
@tool
def {key}():
    """
    Description
    {data[key]["tool_use_case"]}
    """
    input_variables = list(data[key]["inputs"].keys())
    template = data[key]["prompt"].format(
        resume=resume, job_description=job_description, personal_info=personal_info
    )
    prompt_template = PromptTemplate(
        input_variables=input_variables,
        template=template
    )
    chain = prompt_template | llm
    response = chain.invoke(data[key]["inputs"]).content
    return response
'''
    local_vars = {}
    exec(code, globals(), local_vars)
    return local_vars[key]

for key in data.keys():
    globals()[key] = make_tool_func(key)

In [46]:
@tool
def add(a: int, b: int) -> int:
    """Adds two numbers."""
    return a + b

In [51]:
llm1=llm_ds.bind_tools(tools=[add])

In [52]:
response = llm1.invoke("What is 2+2?")
print(response)

content='' additional_kwargs={'reasoning_content': 'Okay, the user is asking, "What is 2+2?" So, I need to figure out how to respond using the tools provided. \n\nLooking at the tools, there\'s an "add" function that takes two integers, a and b, and returns their sum. That seems perfect for this question.\n\nI should call the add function with a=2 and b=2. That should give me the correct result of 4.\n\nI need to format the response as a JSON object within the tool_call XML tags. So, the function name is "add" and the arguments should be an object with a and b both set to 2.\n\nDouble-checking the function parameters to make sure I\'m using the right keys and data types. Everything looks good.\n\nAlright, time to put it all together in the required format.\n', 'tool_calls': [{'id': 'vmcqcgc5w', 'function': {'arguments': '{"a":2,"b":2}', 'name': 'add'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 201, 'prompt_tokens': 134, 'total_tokens': 335, 'completi

In [68]:
llm1=llm_ds.bind_tools(tools=[globals()[key] for key in data.keys()])

In [71]:
llm1.invoke(f"""write a cold mail when resume {resume} and personal _info {personal_info} is given""")

AIMessage(content='', additional_kwargs={'reasoning_content': "Alright, let me try to figure out how to approach this user's query. They've provided a resume for Dinesh Sagar and a personal info section for someone in talent acquisition. They want to write a cold mail, so I need to determine which tool to use.\n\nLooking at the tools available, there's one for when personal info is available and another for when it's not. Since the user provided personal info about the recipient, I should use the cold_mail_and_person_info tool.\n\nNow, I need to structure the arguments correctly. The resume and personal info should be passed as separate strings. I'll make sure to format them properly in JSON, escaping any special characters if necessary.\n\nI should also ensure that the function call is correctly wrapped in the tool_call XML tags as specified. Double-checking the function name and the structure of the arguments is crucial to avoid any errors.\n\nFinally, I'll present the tool_call with

In [ ]:
"""
now  write a langgraph code that uses the tools defined above to create a workflow for generating a cold email based on the resume and personal info provided.
it has one start onde ,input nodes that takes input from the user , three tool modes , and then the  end nodes ,using edged and conditinal edges between llm and tool nodes
the conditional edges are defined by a function that redirects to the tools required based on the output given the input node """
def create_workflow():
    from langchain_core.graph import Graph, Node, StartNode, StopNode
    from langchain_core.graph import Edge, ConditionalEdge

    graph = Graph()

    start_node = StartNode(name="Start")
    input_node = Node(name="Input", input_variables=["resume", "personal_info"])
    tool_nodes = [Node(name=key) for key in data.keys()]
    end_node = StopNode(name="End")

    graph.add_nodes([start_node, input_node, *tool_nodes, end_node])

    graph.add_edges([
        Edge(start_node, input_node),
        *[ConditionalEdge(input_node, tool_node, condition=lambda x: x == tool_node.name) for tool_node in tool_nodes],
        *[Edge(tool_node, end_node) for tool_node in tool_nodes]
    ])

    return graph    

SyntaxError: incomplete input (1825016327.py, line 6)